In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import tree

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,classification_report

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix


In [2]:
df=pd.read_csv(r'C:\Users\Sony\Desktop\python\covid19\another dataset\covid.csv')
df.head()

,id,sex,patient_type,entry_date,date_symptoms,date_died,intubed,pneumonia,age,pregnancy,...,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
0,16169f,2,1,4/5/2020,2/5/2020,9999-99-99,97,2,27,97,...,2,2,2,2,2,2,2,2,1,97
1,1009bf,2,1,19-03-2020,17-03-2020,9999-99-99,97,2,24,97,...,2,2,2,2,2,2,2,99,1,97
2,167386,1,2,6/4/2020,1/4/2020,9999-99-99,2,2,54,2,...,2,2,2,2,1,2,2,99,1,2
3,0b5948,2,2,17-04-2020,10/4/2020,9999-99-99,2,1,30,97,...,2,2,2,2,2,2,2,99,1,2
4,0d01b5,1,2,13-04-2020,13-04-2020,22-04-2020,2,2,60,2,...,2,1,2,1,2,2,2,99,1,2


In [3]:
df=df.drop(['id','entry_date','date_symptoms','date_died'],axis=1)

In [4]:
df.rename(columns={'covid_res':'Test_result'},inplace=True)

In [5]:
df['Test_result'].unique()

array([1, 2, 3], dtype=int64)

In [6]:
#converting numerical features to proper categories
df.sex.replace({1: 'Female', 2: 'Male'}, inplace=True)
df.patient_type.replace({1: 'Outpatient', 2: 'Inpatient'}, inplace=True)
df.intubed.replace({1: 'Yes', 2: 'No',97:'Not Specified', 98:'Not Specified',99:'Not Specified'}, inplace=True)
df.pneumonia.replace({1: 'Yes', 2: 'No', 98:'Not Specified',99:'Not Specified', 97:'Not Specified'}, inplace=True)
df.pregnancy.replace({1: 'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.diabetes.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.copd.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.asthma.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.inmsupr.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.hypertension.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.other_disease.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.cardiovascular.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.obesity.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.renal_chronic.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.tobacco.replace({1:'Yes', 2: 'No', 99:'Not Specified',98:'Not Specified', 97:'Not Specified'}, inplace=True)
df.contact_other_covid.replace({1: 'Yes', 2: 'No', 97:'Not Specified',99:'Not Specified',98:'Not Specified'}, inplace=True)
df.Test_result.replace({1: 'Positive', 2: 'Negative', 3:'Awaiting Results'}, inplace=True)
df.icu.replace({1: 'Yes', 2: 'No', 97:'Not Specified',98:'Not Specified', 99:'Not Specified'}, inplace=True)


In [7]:
# Get names of indexes for which column Stock has value No
indexNames = df[ df['Test_result'] == 'Awaiting Results' ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head()

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,Test_result,icu
0,Male,Outpatient,Not Specified,No,27,Not Specified,No,No,No,No,No,No,No,No,No,No,No,Positive,Not Specified
1,Male,Outpatient,Not Specified,No,24,Not Specified,No,No,No,No,No,No,No,No,No,No,Not Specified,Positive,Not Specified
2,Female,Inpatient,No,No,54,No,No,No,No,No,No,No,No,Yes,No,No,Not Specified,Positive,No
3,Male,Inpatient,No,Yes,30,Not Specified,No,No,No,No,No,No,No,No,No,No,Not Specified,Positive,No
4,Female,Inpatient,No,No,60,No,Yes,No,No,No,Yes,No,Yes,No,No,No,Not Specified,Positive,No


In [8]:
cat_features=df.select_dtypes(include='object').columns.tolist()
print("Categorical Features:",cat_features)
num_features=df.select_dtypes(exclude='object').columns.tolist()
print("Numerical Features:",num_features)

Categorical Features: ['sex', 'patient_type', 'intubed', 'pneumonia', 'pregnancy', 'diabetes', 'copd', 'asthma', 'inmsupr', 'hypertension', 'other_disease', 'cardiovascular', 'obesity', 'renal_chronic', 'tobacco', 'contact_other_covid', 'Test_result', 'icu']
Numerical Features: ['age']


In [9]:

#No null values to handle
df.isna().sum() 
df.isnull().sum()

sex                    0
patient_type           0
intubed                0
pneumonia              0
age                    0
pregnancy              0
diabetes               0
copd                   0
asthma                 0
inmsupr                0
hypertension           0
other_disease          0
cardiovascular         0
obesity                0
renal_chronic          0
tobacco                0
contact_other_covid    0
Test_result            0
icu                    0
dtype: int64

In [10]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
cat_features=df.select_dtypes(include='object').columns.tolist()
le=LabelEncoder()
for col in cat_features:
    df[col]=le.fit_transform(df[col])
    

In [11]:
df['Test_result'].unique()

array([1, 0], dtype=int64)

In [12]:
df.head()

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,Test_result,icu
0,1,1,1,0,27,1,0,0,0,0,0,0,0,0,0,0,0,1,1
1,1,1,1,0,24,1,0,0,0,0,0,0,0,0,0,0,1,1,1
2,0,0,0,0,54,0,0,0,0,0,0,0,0,2,0,0,1,1,0
3,1,0,0,2,30,1,0,0,0,0,0,0,0,0,0,0,1,1,0
4,0,0,0,0,60,0,2,0,0,0,2,0,2,0,0,0,1,1,0


# Splitting data in X and Y

In [15]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


features = ['sex','patient_type','intubed','pneumonia','age','pregnancy','diabetes','copd','asthma','inmsupr','hypertension','other_disease','cardiovascular','obesity','renal_chronic','tobacco','contact_other_covid','icu']
# Separating out the features
x = df.loc[:, features].values

# Separating out the target
y = df.loc[:,['Test_result']].values

# Standardizing the features
x = MinMaxScaler().fit_transform(x)

print(x)

C:\Users\Sony\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


[[1.  1.  0.5 ... 0.  0.  0.5]
 [1.  1.  0.5 ... 0.  0.5 0.5]
 [0.  0.  0.  ... 0.  0.5 0. ]
 ...
 [0.  1.  0.5 ... 0.  0.  0.5]
 [0.  1.  0.5 ... 0.  1.  0.5]
 [0.  1.  0.5 ... 0.  0.5 0.5]]


In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
principalComponents = pca.fit_transform(x)

In [20]:
from sklearn.model_selection import train_test_split
x = df.drop('Test_result', axis = 1)
y = df.Test_result
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.3)

# Logistic regression without CV

In [24]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(class_weight='balanced')
log.fit(x_train, y_train)
log_pred = log.predict(x_test)
log_accuracy = metrics.accuracy_score(y_test, log_pred)
print("log_accuracy",log_accuracy)

log_precision=metrics.precision_score(y_test, log_pred,pos_label=0)
print("log_precision",log_precision)

log_recall=metrics.recall_score(y_test, log_pred,pos_label=0)
print("log_recall",log_recall)

log_f1_score= metrics.f1_score(y_test, log_pred,pos_label=0)
print("log_f1_score",log_f1_score)

print(confusion_matrix(y_test,log_pred))
print(classification_report(y_test,log_pred))

log_accuracy 0.6212076740400779
log_precision 0.6422789144403112
log_recall 0.7260183968462549
log_f1_score 0.6815862370607961
[[60775 22935]
 [33849 32349]]
             precision    recall  f1-score   support

          0       0.64      0.73      0.68     83710
          1       0.59      0.49      0.53     66198

avg / total       0.62      0.62      0.62    149908



# Logistic regression with CV

In [25]:
log_cross_val = cross_val_score(log, x, y, cv=10, scoring='accuracy')
log_cv_accuracy = log_cross_val.mean()
print("log_cv_accuracy",log_cv_accuracy)

log_cross_val_pre = cross_val_score(log, x, y, cv=10, scoring='precision_macro')
log_cv_precision = log_cross_val_pre.mean()
print("log_cv_precision",log_cv_precision)

log_cross_val_re = cross_val_score(log, x, y, cv=10, scoring='recall_macro')
log_cv_recall = log_cross_val_re.mean()
print("log_cv_recall",log_cv_recall)

log_cross_val_f1 = cross_val_score(log, x, y, cv=10, scoring='f1_macro')
log_cv_f1_score = log_cross_val_f1.mean()
print("log_cv_f1_score",log_cv_f1_score)

log_cv_accuracy 0.6199719112510754
log_cv_precision 0.6123895408357136
log_cv_recall 0.6061394248976331
log_cv_f1_score 0.6058493627865866


In [26]:
from sklearn import metrics
from sklearn import tree

dtree = tree.DecisionTreeClassifier(class_weight='balanced')
dtree = dtree.fit(x_train, y_train)

dtree_pred = dtree.predict(x_test)

dtree_accuracy = metrics.accuracy_score(y_test, dtree_pred)
print("dtree_accuracy",dtree_accuracy)

dtree_precision=metrics.precision_score(y_test, dtree_pred,pos_label=0)
print("dtree_precision",dtree_precision)

dtree_recall=metrics.recall_score(y_test, dtree_pred,pos_label=0)
print("dtree_recall",dtree_recall)

dtree_f1_score= metrics.f1_score(y_test, dtree_pred,pos_label=0)
print("dtree_f1_score",dtree_f1_score)

print(confusion_matrix(y_test,dtree_pred))
print(classification_report(y_test,dtree_pred))

dtree_accuracy 0.615590895749393
dtree_precision 0.6363598343858475
dtree_recall 0.727081591207741
dtree_f1_score 0.6787024543639952
[[60864 22846]
 [34780 31418]]
             precision    recall  f1-score   support

          0       0.64      0.73      0.68     83710
          1       0.58      0.47      0.52     66198

avg / total       0.61      0.62      0.61    149908



In [27]:
from sklearn.model_selection import cross_val_score

dtree_cross_val_acc = cross_val_score(dtree, x, y, cv=10, scoring='accuracy')

dtree_cv_accuracy = dtree_cross_val_acc.mean()
print("dtree_cv_accuracy",dtree_cv_accuracy)

dtree_cross_val_pre = cross_val_score(dtree, x, y, cv=10, scoring='precision_macro')
dtree_cv_precision = dtree_cross_val_pre.mean()
print("dtree_cv_precision",dtree_cv_precision)

dtree_cross_val_re = cross_val_score(dtree, x, y, cv=10, scoring='recall_macro')
dtree_cv_recall = dtree_cross_val_re.mean()
print("dtree_cv_recall",dtree_cv_recall)

dtree_cross_val_f1 = cross_val_score(dtree, x, y, cv=10, scoring='f1_macro')
dtree_cv_f1_score = dtree_cross_val_f1.mean()
print("dtree_cv_f1_score",dtree_cv_f1_score)

dtree_cv_accuracy 0.6168399717289313
dtree_cv_precision 0.6092086688073497
dtree_cv_recall 0.6018715336258076
dtree_cv_f1_score 0.6011740547998509


# Random Forest

In [28]:
from sklearn.ensemble import RandomForestClassifier
rnd_for = RandomForestClassifier(class_weight='balanced')
rnd_for.fit(x_train, y_train)
rnd_for_pred = rnd_for.predict(x_test)
rnd_for_accuracy = metrics.accuracy_score(y_test, rnd_for_pred)
print("rnd_for_accuracy",rnd_for_accuracy)

rnd_for_precision=metrics.precision_score(y_test, rnd_for_pred,pos_label=0)
print("rnd_for_precision",rnd_for_precision)

rnd_for_recall=metrics.recall_score(y_test, rnd_for_pred,pos_label=0)
print("rnd_for_recall",rnd_for_recall)

rnd_for_f1_score= metrics.f1_score(y_test, rnd_for_pred,pos_label=0)
print("rnd_for_f1_score",rnd_for_f1_score)

print(confusion_matrix(y_test,rnd_for_pred))
print(classification_report(y_test,rnd_for_pred))

rnd_for_accuracy 0.6156709448461724
rnd_for_precision 0.6351116265584227
rnd_for_recall 0.7326962131167125
rnd_for_f1_score 0.6804228930231526
[[61334 22376]
 [35238 30960]]
             precision    recall  f1-score   support

          0       0.64      0.73      0.68     83710
          1       0.58      0.47      0.52     66198

avg / total       0.61      0.62      0.61    149908



# Random Forest with CV

In [29]:
rnd_for_cross_val = cross_val_score(rnd_for, x, y, cv=10, scoring='accuracy')
rnd_for_cv_accuracy = rnd_for_cross_val.mean()
print("rnd_for_cv_accuracy",rnd_for_cv_accuracy)

rnd_for_cross_val_pre = cross_val_score(rnd_for, x, y, cv=10, scoring='precision_macro')
rnd_for_cv_precision = rnd_for_cross_val_pre.mean()
print("rnd_for_cv_precision",rnd_for_cv_precision)

rnd_for_cross_val_re = cross_val_score(rnd_for, x, y, cv=10, scoring='recall_macro')
rnd_for_cv_recall = rnd_for_cross_val_re.mean()
print("rnd_for_cv_recall",rnd_for_cv_recall)

rnd_for_cross_val_f1 = cross_val_score(rnd_for, x, y, cv=10, scoring='f1_macro')
rnd_for_cv_f1_score = rnd_for_cross_val_f1.mean()
print("rnd_for_cv_f1_score",rnd_for_cv_f1_score)

rnd_for_cv_accuracy 0.6181307734672676
rnd_for_cv_precision 0.6104081437424951
rnd_for_cv_recall 0.601844712164668
rnd_for_cv_f1_score 0.600717046048478


# Gradient Boosting Classifier

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

Grad_boost = GradientBoostingClassifier()
Grad_boost.fit(x_train, y_train)
Grad_boost_pred = Grad_boost.predict(x_test)
Grad_boost_accuracy = metrics.accuracy_score(y_test, Grad_boost_pred)
print("Grad_boost_accuracy",Grad_boost_accuracy)

Grad_boost_precision=metrics.precision_score(y_test, Grad_boost_pred,pos_label=0)
print("Grad_boost_precision",Grad_boost_precision)

Grad_boost_recall=metrics.recall_score(y_test, Grad_boost_pred,pos_label=0)
print("Grad_boost_recall",Grad_boost_recall)

Grad_boost_f1_score= metrics.f1_score(y_test, Grad_boost_pred,pos_label=0)
print("Grad_boost_f1_score",Grad_boost_f1_score)

print(confusion_matrix(y_test,Grad_boost_pred))
print(classification_report(y_test,Grad_boost_pred))

Grad_boost_accuracy 0.6405528724284227
Grad_boost_precision 0.6336961199168041
Grad_boost_recall 0.844403297097121
Grad_boost_f1_score 0.7240312618435474
[[70685 13025]
 [40859 25339]]
             precision    recall  f1-score   support

          0       0.63      0.84      0.72     83710
          1       0.66      0.38      0.48     66198

avg / total       0.65      0.64      0.62    149908



# Gradient Boosting Classifier with Cross Validation

In [19]:
Grad_boost_cross_val = cross_val_score(Grad_boost, x, y, cv=10, scoring='accuracy')
Grad_boost_cv_accuracy = Grad_boost_cross_val.mean()
print(Grad_boost_cv_accuracy)

Grad_boost_cross_val_pre = cross_val_score(Grad_boost, x, y, cv=10, scoring='precision_macro')
Grad_boost_cv_precision = Grad_boost_cross_val_pre.mean()
print(Grad_boost_cv_precision)

Grad_boost_cross_val_re = cross_val_score(Grad_boost, x, y, cv=10, scoring='recall_macro')
Grad_boost_cv_recall = Grad_boost_cross_val_re.mean()
print(Grad_boost_cv_recall)

Grad_boost_cross_val_f1 = cross_val_score(Grad_boost, x, y, cv=10, scoring='f1_macro')
Grad_boost_cv_f1_score = Grad_boost_cross_val_f1.mean()
print(Grad_boost_cv_f1_score)

0.6399762318052351
0.6467705007359406
0.6128142671574783
0.603300823261115


# Multilayer Perceptron

In [21]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(max_iter=200,activation='logistic')
MLP.fit(x_train, y_train)
MLP_pred = MLP.predict(x_test)
MLP_accuracy = metrics.accuracy_score(y_test, MLP_pred)
print("MLP_accuracy",MLP_accuracy)

MLP_precision=metrics.precision_score(y_test, MLP_pred,pos_label=0)
print("MLP_precision",MLP_precision)

MLP_recall=metrics.recall_score(y_test, MLP_pred,pos_label=0)
print("MLP_recall",MLP_recall)

MLP_f1_score= metrics.f1_score(y_test, MLP_pred,pos_label=0)
print("MLP_f1_score",MLP_f1_score)

print(confusion_matrix(y_test,MLP_pred))
print(classification_report(y_test,MLP_pred))

MLP_accuracy 0.6411265576220082
MLP_precision 0.6351669482083169
MLP_recall 0.8426994420866911
MLP_f1_score 0.7243616018362914
[[70689 13195]
 [40603 25421]]
             precision    recall  f1-score   support

          0       0.64      0.84      0.72     83884
          1       0.66      0.39      0.49     66024

avg / total       0.65      0.64      0.62    149908



# Multilayer Perceptron with Cross Validation

In [22]:
MLP_cross_val = cross_val_score(MLP, x, y, cv=10, scoring='accuracy')
MLP_cv_accuracy = MLP_cross_val.mean()
print("MLP_cv_accuracy",MLP_cv_accuracy)

MLP_cross_val_pre = cross_val_score(MLP, x, y, cv=10, scoring='precision_macro')
MLP_cv_precision = MLP_cross_val_pre.mean()
print("MLP_cv_precision",MLP_cv_precision)

MLP_cross_val_re = cross_val_score(MLP, x, y, cv=10, scoring='recall_macro')
MLP_cv_recall = MLP_cross_val_re.mean()
print("MLP_cv_recall",MLP_cv_recall)

MLP_cross_val_f1 = cross_val_score(MLP, x, y, cv=10, scoring='f1_macro')
MLP_cv_f1_score = MLP_cross_val_f1.mean()
print("MLP_cv_f1_score",MLP_cv_f1_score)

MLP_cv_accuracy 0.639712071908395
MLP_cv_precision 0.6461451129944953
MLP_cv_recall 0.6153165229291576
MLP_cv_f1_score 0.6056863019545992
